<a href="https://colab.research.google.com/github/VSharma27/CCFD1/blob/main/Source_Code_PPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os # clear screen
import re # for scanning input data
import pandas as pd


create a class of task

In [2]:
class Task(object):
  def __init__(self, description, activity, predecessors, days):
    self.description = description
    self.activity = activity.upper()
    self.predecessors = predecessors
    self.days=days
    self.earlyStart=0
    self.earliestFinish=0
    self.successors=[]
    self.latestStart=0
    self.latestFinish=0
    self.slack=0
    self.critical=''
  

  def computeSlack(self):
    self.slack=self.latestFinish-self.earliestFinish
    if self.slack>0:
      self.critical='NO'
    else:
      self.critical ='YES'


Compute slack for a task
Slack is the difference between early finish and early start or
late finish and late start


The functions

In [3]:


def readData(): #Get data and return a pandas dataframe
  myFile='data.xlsx'
  sheet=pd.read_excel(myFile, sheet_name='Sheet1')
  return (sheet)

def createTask(mydata):# create task object from pandas data frame
  taskObject=[]

  for i in range(len(mydata)):
    taskObject.append(Task(mydata['DESCRIPTION'][i],mydata['ACTIVITY'][i], mydata['PREDECESSORS'][i],mydata['DAYS'][i]))
  
  return (taskObject)

def forwardPass(taskObject):
  for task in taskObject:
    if type(task.predecessors) is str: #type string
      task.predecessors= task.predecessors.upper()
      ef=[]
      for j in task.predecessors:
          for t in taskObject:
              if t.activity==j:
                  ef.append(t.earliestFinish)
          task.earlyStart=max(ef)
      del ef
    else:
        task.earlyStart=0

    task.earliestFinish=task.earlyStart+task.days
  
def backwardPass(taskObject):
  pred=[]
  eF=[]

  for task in taskObject:
    if type(task.predecessors) ==str:
      for j in task.predecessors:
        pattern = re.compile(r'[A-Z]')
        match=pattern.finditer(j)
        for r in match:
          pred.append(j)
          for m in taskObject:
            if m.activity ==j:
              m.successors.append(task.activity)

    eF.append(task.earliestFinish)



    for task in reversed(taskObject):
      if task.activity not in pred:
        task.latestFinish =max(eF)
      else:
        minLs =[]
        for x in task.successors:
          for t in (taskObject):
            if t.activity==x:
              minLs.append(t.latestStart)
        task.latestFinish =min(minLs)
        del minLs
      
      task.latestStart=task.latestFinish-task.days

def slack(taskObject):
  for task in taskObject:
    task.computeSlack()

def updateDataFrame(df,TaskObject):
  df2=pd.DataFrame({
  'DESCRIPTION':df['DESCRIPTION'],
  'ACTIVITY':df['ACTIVITY'],
  'PREDECESSORS':df['PREDECESSORS'],
  'DAYS':df['DAYS'],
  'ES':pd.Series([task.earlyStart for task in TaskObject]),
  'EF':pd.Series([task.earliestFinish for task in TaskObject]),
  'LS':pd.Series([task.latestStart for task in TaskObject]),
  'LF':pd.Series([task.latestFinish for task in TaskObject]),
  'SLACK':pd.Series([task.slack for task in TaskObject]),
  'CRITICAL?':pd.Series([task.critical for task in TaskObject])
  })
  return (df2)

def main():
  os.system('clear')
  df=readData()
  taskObject=createTask(df)
  forwardPass(taskObject)
  backwardPass(taskObject)
  slack(taskObject)

  finaldf=updateDataFrame(df,taskObject)
  print(finaldf)
  print('\n File saved to cpm.xlsx\n')
  finaldf.to_excel('cpm.xlsx', index = False)



In [4]:
main()

  DESCRIPTION ACTIVITY PREDECESSORS  DAYS  ES  EF  LS  LF  SLACK CRITICAL?
0  Activity 1        A          NaN     6   0   6   3   9      3        NO
1  Activity 2        B          NaN     9   0   9   0   9      0       YES
2  Activity 3        C          A,B    10   9  19   9  19      0       YES
3  Activity 4        D            C     8  19  27  19  27      0       YES
4  Activity 5        E            C     5  19  24  22  27      3        NO
5  Activity 6        F          E,D     6  27  33  27  33      0       YES
6  Activity 7        G            F     5  33  38  36  41      3        NO
7  Activity 8        H            F     8  33  41  33  41      0       YES

 File saved to cpm.xlsx

